In [1]:
import sys
#!{sys.executable} -m pip install pafy
#!{sys.executable} -m pip install youtube-dl
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install facenet-pytorch

    100% |################################| 1.9MB 278kB/s eta 0:00:01
    100% |################################| 1.9MB 273kB/s eta 0:00:01


In [1]:
import time
from os import getenv
import os
import io
from dotenv import load_dotenv
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import ipywidgets as widgets
import traitlets
import PIL
from PIL import ImageDraw
from IPython.display import display, Image
from jetcam.utils import bgr8_to_jpeg
import IPython
from jetcam.usb_camera import USBCamera
from facenet_pytorch import MTCNN
from IPython.display import Image
from my_heatmap_generator import HMap
import cv2

device = torch.device('cuda')
mtcnn = MTCNN(keep_all=True, device=device)
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)
model = model.to(device)
model.load_state_dict(torch.load('./model/my_model(1).pth'))
model.eval()
print("model loaded")
outframe = None
url = "https://youtu.be/7PYzSXHd6U4"
load_dotenv()

#defaults
active = "Live"
default_radius_size = getenv('DEFAULT_RADIUS')
default_duration = getenv('DEFAULT_DURATION')
radius = int(getenv('DEFAULT_RADIUS'))
duration = int(getenv('DEFAULT_DURATION'))

def on_value_change(change):
    radius = radius_slider.value
    duration = duration_slider.value

duration_slider = widgets.IntSlider(
    value=duration,
    min = 2,
    max = 30,
    step=1,
    description='Airborne Duration:',
    continuous_update=True,
    orientation='horizontal'
)

radius_slider = widgets.IntSlider(
    value=duration,
    min = 10,
    max = 200,
    step=1,
    description='Radius:',
    continuous_update=True,
    orientation='horizontal'
)

model loaded


In [2]:
hmap = None
camera = USBCamera(width=224, height=224, capture_fps=30, capture_device=0)
camera.running = True
print("camera created")

def preprocess(image):
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

image_widget = widgets.Image(format='jpeg', width=224, height=224)

def execute(change):
    global hmap
    frame = change['new']
    frame = PIL.Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    width, height = frame.size[:2]
    boxes, _ = mtcnn.detect(frame)
    face_only = None
    if boxes is not None:
        for box in boxes:
            rectangle = (box[0]-20, box[1]-20, box[2]+20, box[3]+20)
            face_only = frame.crop(rectangle)
            face_only = face_only.resize((224, 224))
            processed = preprocess(face_only)
            output = model(processed)
            output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
            category_index = output.argmax()
            hmap = HMap(int(height), int(width), int(box[0]), int(box[1]), category_index)
            tmp = hmap.apply_color_map(frame, radius, duration)
            if type(tmp) is not PIL.Image.Image:
                frame = PIL.Image.fromarray(tmp, "RGB")
    buf = io.BytesIO()
    frame.save(buf, format='JPEG')
    image_widget.value = buf.getvalue()

duration_slider.observe(on_value_change)
radius_slider.observe(on_value_change)
camera.observe(execute, names='value')
display_handle = display(image_widget, widgets.HBox([duration_slider, radius_slider]))

while True:
    execute({'new': camera.value})
    on_value_change({'new': duration_slider.value})

camera created


Image(value=b'', format='jpeg', height='224', width='224')

8.392333984375e-05
3.314018249511719e-05
3.838539123535156e-053.695487976074219e-05

1.4066696166992188e-05
9.059906005859375e-06
1.621246337890625e-059.5367431640625e-06

1.430511474609375e-05
1.2159347534179688e-05
1.9073486328125e-05
9.5367431640625e-06
3.314018249511719e-05
3.0994415283203125e-05
4.553794860839844e-05
1.1920928955078125e-05
1.5735626220703125e-05
4.267692565917969e-05
2.7179718017578125e-05
1.3589859008789062e-05
1.52587890625e-05
1.049041748046875e-05
1.3589859008789062e-05
2.7179718017578125e-05
1.6450881958007812e-05
1.2874603271484375e-05
2.6226043701171875e-051.0728836059570312e-05

1.5497207641601562e-05
3.4332275390625e-05
1.4543533325195312e-05
9.5367431640625e-06
1.3589859008789062e-05
9.775161743164062e-06
1.430511474609375e-059.775161743164062e-06

1.9073486328125e-05
3.123283386230469e-05
1.4543533325195312e-05
1.1205673217773438e-05
1.33514404296875e-05
1.1205673217773438e-05
1.2159347534179688e-05
3.266334533691406e-05
3.0517578125e-05


KeyboardInterrupt: 

1.5735626220703125e-05
3.361701965332031e-05
2.765655517578125e-05
1.5497207641601562e-05
1.3113021850585938e-05
1.4543533325195312e-05
3.910064697265625e-05
1.5020370483398438e-05
3.7670135498046875e-05
1.33514404296875e-05
1.430511474609375e-05
1.4066696166992188e-05
1.430511474609375e-05
4.696846008300781e-05
3.528594970703125e-05
1.3113021850585938e-05
1.3828277587890625e-05
1.2636184692382812e-05
1.4066696166992188e-05
1.430511474609375e-05
1.52587890625e-05
1.3828277587890625e-05
5.9604644775390625e-05
1.7881393432617188e-05
1.2874603271484375e-05
1.33514404296875e-05
1.4781951904296875e-05
1.4066696166992188e-05
1.4781951904296875e-05
1.5020370483398438e-05
2.956390380859375e-05
3.457069396972656e-05
